In [ ]:
import os, sys
current_dir = os.path.dirname(os.getcwd())
sys.path.append(current_dir)
from common_imports import *

In [ ]:
models = os.listdir(diffusers_models_path)
print(models)
model_path = os.path.join(diffusers_models_path, models[1])
device = "cuda"
pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None)
pipe = pipe.to(device, torch.float16)
# print pytorch version
print(torch.__version__)

In [ ]:
def get_embed(prompt):
    tokens = pipe.tokenizer.tokenize(prompt)
    print(tokens)
    text_inputs = pipe.tokenizer(
                prompt,
                padding="max_length",
                max_length=pipe.tokenizer.model_max_length,
                truncation=True,
                return_tensors="pt",
            )    
    prompt_embeds = pipe.text_encoder(
                text_inputs.input_ids.to(device))
    return prompt_embeds[0]

In [ ]:

prompt = "solid blue ball aesthetic"
neg_prompt = ""

cond_emb = get_embed(prompt)
uncond_emb = get_embed(neg_prompt)

print(cond_emb.min(), cond_emb.max())

norm_cond_emb = (cond_emb - cond_emb.min()) / (cond_emb.max() - cond_emb.min())
print(norm_cond_emb.min(), norm_cond_emb.max())
#plot cond_emb[0]

emd = norm_cond_emb[0].cpu().detach().numpy()
emd = emd[0:5].reshape(5,24,32)


#print(emd[0,19])
#print(emd[21,9])
# add title
plt.figure(figsize=(15,5))
for i in range(emd.shape[0]):
    plt.subplot(2,4,i+1)
    plt.imshow(emd[i], cmap='gray')
    plt.title(f"Embedding {i}")
plt.show()

In [ ]:
cond_emb = get_embed(prompt)
print(cond_emb.shape)
print(cond_emb.min(), cond_emb.max())
#cond_emb[0,9:77,0:768] = 0
#cond_emb[0,19,0] = 1.0
generator = torch.Generator(device="cuda").manual_seed(30)
image = pipe(prompt_embeds=cond_emb,
             generator=generator,
             num_inference_steps = 20,
             height = 512,
             width = 512,        
             guidance_scale = 10).images[0]
#plot the image
plt.figure(figsize=(4,6))
plt.imshow(image, interpolation="sinc")
plt.show()
